#### <center>Rendimiento escolar</center>

#### HITO 1

#### Preliminares

#### Aspectos (Librerías)

In [95]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

#importar funciones creadas para analisis


from funciones_1 import*

#### Consideraciones

#### Data Frame

In [96]:
df=pd.read_csv("students.csv")

In [97]:
df.head()

,Unnamed: 0,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,0,GP,F,nulidade,U,GT3,A,4,4,at_home,...,4,3,"""4""",1,1,"""3""",6,5,6,6
1,1,GP,F,"""17""",U,GT3,T,1,1,at_home,...,5,3,"""3""",1,1,"""3""",4,5,5,6
2,2,GP,F,"""15""",U,LE3,T,1,1,at_home,...,4,3,"""2""",2,3,"""3""",10,zero,8,10
3,3,GP,F,"""15""",U,GT3,T,4,2,health,...,3,2,"""2""",1,1,"""5""",2,15,14,15
4,4,GP,F,sem validade,U,GT3,T,3,3,other,...,4,3,"""2""",1,2,"""5""",4,6,10,10


In [98]:
df = df.drop(columns='Unnamed: 0') # se elimina columna sin nombre

In [99]:
df.sample(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
68,GP,F,"""15""",R,LE3,T,2,2,health,services,...,4,1,"""3""",1,3,"""4""",2,8,9,8
180,GP,M,"""16""",U,GT3,T,4,3,teacher,other,...,3,4,"""3""",2,3,"""3""",10,9,8,8
179,GP,M,"""17""",U,GT3,T,4,3,other,other,...,5,2,"""3""",1,1,"""2""",4,10,10,11
75,GP,M,"""15""",U,GT3,T,4,3,teacher,other,...,4,3,"""3""",2,3,"""5""",6,9,9,10
106,GP,F,"""15""",U,zero,T,2,2,other,other,...,5,1,"""2""",1,1,"""3""",8,7,8,8


#### HITO2

#### Inspección visual del vector objetivo.

In [100]:
df['G3'].value_counts()

10          55
11          47
0           37
15          33
8           32
13          31
12          31
9           27
14          27
16          16
6           15
18          12
7            9
5            7
17           6
19           5
nulidade     3
20           1
4            1
Name: G3, dtype: int64

In [101]:
#G3 es de tipo object y presenta 3 valores nulos (nulidade)

#### Inspección visual de las variables

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      395 non-null    object
 1   sex         395 non-null    object
 2   age         395 non-null    object
 3   address     395 non-null    object
 4   famsize     395 non-null    object
 5   Pstatus     395 non-null    object
 6   Medu        395 non-null    object
 7   Fedu        395 non-null    object
 8   Mjob        395 non-null    object
 9   Fjob        395 non-null    object
 10  reason      395 non-null    object
 11  guardian    395 non-null    object
 12  traveltime  395 non-null    object
 13  studytime   395 non-null    object
 14  failures    395 non-null    object
 15  schoolsup   395 non-null    object
 16  famsup      395 non-null    object
 17  paid        395 non-null    object
 18  activities  395 non-null    object
 19  nursery     395 non-null    object
 20  higher    

In [103]:
#Casi todas las variables son tipo object, a excepción de G2. Se inspeccionará detalladamente estos tipos

#### Inspección de datos perdidos en las variables

In [104]:
variables_insp(df)


 school
GP              345
MS               45
sem validade      2
zero              2
nulidade          1
Name: school, dtype: int64

 sex
F               206
M               186
nulidade          1
zero              1
sem validade      1
Name: sex, dtype: int64

 age
"16"            101
"17"             98
"15"             82
"18"             81
"19"             24
nulidade          3
"20"              3
sem validade      1
"22"              1
"21"              1
Name: age, dtype: int64

 address
U               305
R                88
sem validade      2
Name: address, dtype: int64

 famsize
GT3             278
LE3             113
sem validade      2
zero              2
Name: famsize, dtype: int64

 Pstatus
T               349
A                40
sem validade      4
nulidade          1
zero              1
Name: Pstatus, dtype: int64

 Medu
4               129
2               103
3                98
1                59
0                 3
sem validade      2
nulidade          1
Nam

In [105]:
#Existen varias variables con valores tipo 'sem validade', 'zero' o 'nulidade'. Estas opciones no aportan información de la variable,
#por lo cual, se decide eliminar las columnas que las contengan. 

In [106]:
df=df.replace({"sem validade": np.nan, "nulidade": np.nan, "zero": np.nan})

In [107]:
## ELiminamos los NaN, es decir las filas que tengan NA. Con este cambio nos quedan 284 filas con información.
df=df.dropna()

In [108]:
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
1,GP,F,"""17""",U,GT3,T,1,1,at_home,other,...,5,3,"""3""",1,1,"""3""",4,5,5,6
3,GP,F,"""15""",U,GT3,T,4,2,health,services,...,3,2,"""2""",1,1,"""5""",2,15,14,15
5,GP,M,"""16""",U,LE3,T,4,3,services,other,...,5,4,"""2""",1,2,"""5""",10,15,15,15
6,GP,M,"""16""",U,LE3,T,2,2,other,other,...,4,4,"""4""",1,1,"""3""",0,12,12,11
7,GP,F,"""17""",U,GT3,A,4,4,other,teacher,...,4,1,"""4""",1,1,"""1""",6,6,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,"""20""",U,LE3,A,2,2,services,services,...,5,5,"""4""",4,5,"""4""",11,9,9,9
391,MS,M,"""17""",U,LE3,T,3,1,services,services,...,2,4,"""5""",3,4,"""2""",3,14,16,16
392,MS,M,"""21""",R,GT3,T,1,1,other,other,...,5,5,"""3""",3,3,"""3""",3,10,8,7
393,MS,M,"""18""",R,LE3,T,3,2,services,other,...,4,4,"""1""",3,4,"""5""",0,11,12,10


In [109]:
#Existen variables que deberían ser numericas (float, integer) pero al tener estos valores string están como object

In [110]:
variables_insp(df)


 school
GP    247
MS     37
Name: school, dtype: int64

 sex
F    147
M    137
Name: sex, dtype: int64

 age
"16"    74
"18"    70
"15"    61
"17"    60
"19"    15
"20"     3
"21"     1
Name: age, dtype: int64

 address
U    214
R     70
Name: address, dtype: int64

 famsize
GT3    209
LE3     75
Name: famsize, dtype: int64

 Pstatus
T    257
A     27
Name: Pstatus, dtype: int64

 Medu
4    98
2    71
3    70
1    44
0     1
Name: Medu, dtype: int64

 Fedu
2    77
3    76
4    68
1    62
0     1
Name: Fedu, dtype: int64

 Mjob
other       103
services     68
teacher      44
at_home      43
health       26
Name: Mjob, dtype: int64

 Fjob
other       159
services     80
teacher      21
health       13
at_home      11
Name: Fjob, dtype: int64

 reason
course        102
home           78
reputation     77
other          27
Name: reason, dtype: int64

 guardian
mother    200
father     60
other      24
Name: guardian, dtype: int64

 traveltime
1    177
2     84
3     18
4      5
Name: trav

In [111]:
#Según el resumen de opciones de cada variable se puede apreciar que las que deberían ser numéricas son:
# Age, Medu, Fedu, traveltime, studytime, failures, famrel, freetime, goout, Dalc, Walc, health, absences, G1, G3
# Como los valores de estas variables son números pero están en como texto, se modifican de tipo y no se recodifican. 
#Primero se modifican valores con comillas ()

In [112]:
df['age'] = df['age'].str.replace('"','')

In [113]:
df['health'] = df['health'].str.replace('"','')

In [114]:
df['goout'] = df['goout'].str.replace('"','')

In [115]:
#Ahora se cambian las variables a numerico (int)

In [116]:

df[['age', 'Medu', 'Fedu','traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G3']] = df[['age', 'Medu', 'Fedu','traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G3']].astype('int64')


In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 1 to 394
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      284 non-null    object
 1   sex         284 non-null    object
 2   age         284 non-null    int64 
 3   address     284 non-null    object
 4   famsize     284 non-null    object
 5   Pstatus     284 non-null    object
 6   Medu        284 non-null    int64 
 7   Fedu        284 non-null    int64 
 8   Mjob        284 non-null    object
 9   Fjob        284 non-null    object
 10  reason      284 non-null    object
 11  guardian    284 non-null    object
 12  traveltime  284 non-null    int64 
 13  studytime   284 non-null    int64 
 14  failures    284 non-null    int64 
 15  schoolsup   284 non-null    object
 16  famsup      284 non-null    object
 17  paid        284 non-null    object
 18  activities  284 non-null    object
 19  nursery     284 non-null    object
 20  higher    

In [118]:
# Se observan ahora la frecuencia de las variables discretas recién convertidas

In [119]:
retorna_valores(df)

Freecuencia de variables discretas


16    74
18    70
15    61
17    60
19    15
20     3
21     1
Name: age, dtype: int64

None


4    98
2    71
3    70
1    44
0     1
Name: Medu, dtype: int64

None


...

None


9     38
10    34
12    32
13    25
11    23
8     22
15    20
7     17
14    16
6     11
16    11
0     10
5      9
18     7
17     5
19     3
4      1
Name: G2, dtype: int64

None


10    40
11    32
0     26
12    24
13    22
8     21
9     21
15    20
14    20
6     13
16    11
7      9
18     7
17     6
5      5
19     5
20     1
4      1
Name: G3, dtype: int64

None


In [120]:
# Ahora se trabajarán las variables categóricas.

In [121]:
variables_insp(df)


 school
GP    247
MS     37
Name: school, dtype: int64

 sex
F    147
M    137
Name: sex, dtype: int64

 address
U    214
R     70
Name: address, dtype: int64

 famsize
GT3    209
LE3     75
Name: famsize, dtype: int64

 Pstatus
T    257
A     27
Name: Pstatus, dtype: int64

 Mjob
other       103
services     68
teacher      44
at_home      43
health       26
Name: Mjob, dtype: int64

 Fjob
other       159
services     80
teacher      21
health       13
at_home      11
Name: Fjob, dtype: int64

 reason
course        102
home           78
reputation     77
other          27
Name: reason, dtype: int64

 guardian
mother    200
father     60
other      24
Name: guardian, dtype: int64

 schoolsup
no     246
yes     38
Name: schoolsup, dtype: int64

 famsup
yes    174
no     110
Name: famsup, dtype: int64

 paid
no     158
yes    126
Name: paid, dtype: int64

 activities
yes    148
no     136
Name: activities, dtype: int64

 nursery
yes    223
no      61
Name: nursery, dtype: int64

 higher

In [122]:
# Se procede a la binarización, para ello se toma la opción con menor frecuencia como valor 1 para menor costo computacional

In [123]:
df['is_Male']= np.where(df['sex'] == 'M',1,0 )
df['is_school_MS']= np.where(df['school'] == 'MS',1,0 )
df['is_address_R']= np.where(df['address'] == 'R',1,0 )
df['is_famsize_LE3']= np.where(df['famsize'] == 'LE3',1,0 )
df['is_Pstatus_A']= np.where(df['Pstatus'] == 'A',1,0 )

df['is_schoolsup']= np.where(df['schoolsup'] == 'yes',1,0 )
df['not_famsup']= np.where(df['famsup'] == 'no',1,0 )
df['is_paid']= np.where(df['paid'] == 'yes',1,0 )
df['not_activities']= np.where(df['activities'] == 'no',1,0 )
df['not_nursery']= np.where(df['nursery'] == 'no',1,0 )
df['not_higher']= np.where(df['higher'] == 'no',1,0 )
df['not_internet']= np.where(df['schoolsup'] == 'no',1,0 )
df['is_romantic']= np.where(df['romantic'] == 'yes',1,0 )






In [124]:
##Las variables de más de dos opciones se binarizan de la siguiente manera.
#Trabajo de la madre se separa en si madre trabaja fuera de casa o no trabaja fuera de casa
df['not_Mjob']= np.where(df['Mjob'] == 'at_home',1,0 )



In [125]:
#Trabajo del padre se separa en si padre trabaja fuera de casa o no trabaja fuera de casa

df['not_Fjob']= np.where(df['Fjob'] == 'at_home',1,0 )

In [126]:
#: Razón para escoger la escuela se separó por cada opcion (menos opción otros)
df['is_reason_reputation']= np.where(df['reason'] == 'reputation', 1,0 ) 
df['is_reason_course']= np.where(df['reason'] == 'course', 1,0 ) 
df['is_reason_home']= np.where(df['reason'] == 'home', 1,0 ) 

In [127]:
# Apoderado del estudiante se separó por padres o no padres

df['is_guardian_notparent']= np.where(df['guardian'] == 'other',1,0 )

#### HITO 3

#### Modelación descriptiva

In [128]:
# Tenemos nuestras variables Binarizadas
df.sample(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,not_nursery,not_higher,not_internet,is_romantic,not_Mjob,not_Fjob,is_reason_reputation,is_reason_course,is_reason_home,is_guardian_notparent
100,GP,M,16,U,GT3,T,4,4,services,services,...,0,0,0,0,0,0,0,0,0,0
19,GP,M,16,U,LE3,T,4,3,health,other,...,0,0,1,0,0,0,0,0,1,0
346,GP,M,18,R,GT3,T,4,3,teacher,services,...,0,0,1,1,0,0,0,1,0,0
354,MS,M,17,R,GT3,T,4,3,services,other,...,1,0,1,1,0,0,0,0,1,0
228,GP,M,18,U,LE3,T,2,1,at_home,other,...,0,0,0,1,1,0,0,1,0,0


In [129]:
df.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,not_nursery,not_higher,not_internet,is_romantic,not_Mjob,not_Fjob,is_reason_reputation,is_reason_course,is_reason_home,is_guardian_notparent
count,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.00000,284.000000,284.000000,...,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000
mean,16.707746,2.774648,2.521127,1.475352,2.049296,0.334507,3.940141,3.18662,3.084507,1.482394,...,0.214789,0.052817,0.866197,0.334507,0.151408,0.038732,0.271127,0.359155,0.274648,0.084507
std,1.284238,1.095409,1.091038,0.695334,0.843411,0.730994,0.881659,1.02117,1.076527,0.879512,...,0.411401,0.224063,0.341041,0.472651,0.359079,0.193297,0.445326,0.480600,0.447125,0.278638
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.00000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.00000,2.000000,1.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,17.000000,3.000000,3.000000,1.000000,2.000000,0.000000,4.000000,3.00000,3.000000,1.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.00000,4.000000,2.000000,...,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
max,21.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.00000,5.000000,5.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [130]:
#Generamos un modelo con todos los regresores, para ello no se consideran las variables categóricas 
#(Se sacan las variables a predecir G1, G2, G3)

In [131]:
v_no_categoricas = []
for n, i in enumerate(df):
    if (i !='G3' and df[i].dtypes !="object" and i !='G1' and i !='G2'):
        v_no_categoricas.append(i)
v_independientes = "+".join(v_no_categoricas)
v_independientes

'age+Medu+Fedu+traveltime+studytime+failures+famrel+freetime+goout+Dalc+Walc+health+absences+is_Male+is_school_MS+is_address_R+is_famsize_LE3+is_Pstatus_A+is_schoolsup+not_famsup+is_paid+not_activities+not_nursery+not_higher+not_internet+is_romantic+not_Mjob+not_Fjob+is_reason_reputation+is_reason_course+is_reason_home+is_guardian_notparent'

In [132]:
#Modelo con todos los regresores
modelo_saturado = smf.ols('G3 ~ '+v_independientes, data=df)
modelo_saturado = modelo_saturado.fit()
modelo_saturado.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Results: Ordinary least squares
=====================================================================
Model:               OLS               Adj. R-squared:      0.194    
Dependent Variable:  G3                AIC:                 1637.4969
Date:                2022-11-23 05:32  BIC:                 1754.2641
No. Observations:    284               Log-Likelihood:      -786.75  
Df Model:            31                F-statistic:         3.191    
Df Residuals:        252               Prob (F-statistic):  2.26e-07 
R-squared:           0.282             Scale:               16.813   
---------------------------------------------------------------------
                       Coef.  Std.Err.    t    P>|t|   [0.025  0.975]
---------------------------------------------------------------------
Intercept              9.2111   3.0976  2.9736 0.0032  3.1106 15.3116
age                   -0.2210   0.2589 -0.8535 0.3942 -0.7308  0.2889
Medu                   0.4194   0.3391  1.2368 0.2173 -0.2484  1.0872
Fedu                  -0.1127   0.3059 -0.3685 0.7128 -0.7151  0.4897
traveltime            -0.1672   0.4052 -0.4126 0.6802 -0.9651  0.6307
studytime              0.3652   0.3361  1.0866 0.2783 -0.2967  1.0271
failures              -1.3377   0.3947 -3.3890 0.0008 -2.1151 -0.5603
famrel                 0.4657   0.2931  1.5890 0.1133 -0.1115  1.0429
freetime               0.4038   0.2711  1.4892 0.1377 -0.1302  0.9378
goout                 -1.1675   0.2780 -4.2001 0.0000 -1.7149 -0.6201
Dalc                  -0.5294   0.3867 -1.3688 0.1723 -1.2910  0.2323
Walc                   0.7088   0.2897  2.4464 0.0151  0.1382  1.2793
health                -0.3597   0.1965 -1.8305 0.0684 -0.7467  0.0273
absences               0.0337   0.0338  0.9978 0.3193 -0.0328  0.1002
is_Male                0.7956   0.5990  1.3283 0.1853 -0.3841  1.9753
is_school_MS           0.9561   0.9192  1.0401 0.2993 -0.8543  2.7665
is_address_R          -1.1028   0.6739 -1.6364 0.1030 -2.4301  0.2244
is_famsize_LE3         0.3628   0.6017  0.6030 0.5471 -0.8222  1.5478
is_Pstatus_A           0.8800   0.8959  0.9823 0.3269 -0.8843  2.6444
is_schoolsup           4.1294   1.5159  2.7241 0.0069  1.1440  7.1149
not_famsup             0.8025   0.5742  1.3977 0.1634 -0.3283  1.9333
is_paid                0.2692   0.5532  0.4865 0.6270 -0.8204  1.3587
not_activities         0.6394   0.5123  1.2481 0.2131 -0.3695  1.6482
not_nursery           -0.0590   0.6363 -0.0928 0.9262 -1.3121  1.1941
not_higher            -1.6500   1.2276 -1.3441 0.1801 -4.0676  0.7676
not_internet           5.0816   1.6753  3.0333 0.0027  1.7823  8.3809
is_romantic           -0.7208   0.5535 -1.3024 0.1940 -1.8108  0.3692
not_Mjob              -0.4139   0.7926 -0.5222 0.6020 -1.9747  1.1470
not_Fjob               0.8019   1.3439  0.5967 0.5513 -1.8449  3.4486
is_reason_reputation  -0.3465   0.9869 -0.3511 0.7258 -2.2901  1.5972
is_reason_course      -1.5047   0.9309 -1.6164 0.1073 -3.3381  0.3286
is_reason_home        -1.1568   0.9832 -1.1766 0.2405 -3.0930  0.7795
is_guardian_notparent -0.5613   1.0818 -0.5189 0.6043 -2.6917  1.5691
---------------------------------------------------------------------
Omnibus:           11.116     Durbin-Watson:        1.872            
Prob(Omnibus):     0.004      Jarque-Bera (JB):     11.665           
Skew:              -0.496     Prob(JB):             0.003            
Kurtosis:          3.006      Condition No.:        25897555596586012
=====================================================================
* The condition number is large (3e+16). This might indicate
strong multicollinearity or other numerical problems.
"""

In [133]:
# El modelo saturado tiene un  R^2 de 0.28 % de la variable G3, esto es muy bajo. Si consideramos una significancia p<=0.05 podríamos decir
# que las variables que aportan significativamente a la predicción del valor de la variable G3 son: failures, goout,Walc,
# is_schoolsup y not_internet


In [134]:
#Se estimara el modelo saturado para G1 y G2 para contrastar la información anterior

In [135]:
#Modelo con todos los regresores
modelo_saturadog1 = smf.ols('G1 ~ '+v_independientes, data=df)
modelo_saturadog1 = modelo_saturadog1.fit()
modelo_saturadog1.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Results: Ordinary least squares
=====================================================================
Model:               OLS               Adj. R-squared:      0.185    
Dependent Variable:  G1                AIC:                 1454.2402
Date:                2022-11-23 05:32  BIC:                 1571.0074
No. Observations:    284               Log-Likelihood:      -695.12  
Df Model:            31                F-statistic:         3.070    
Df Residuals:        252               Prob (F-statistic):  5.90e-07 
R-squared:           0.274             Scale:               8.8188   
---------------------------------------------------------------------
                       Coef.  Std.Err.    t    P>|t|   [0.025  0.975]
---------------------------------------------------------------------
Intercept              5.7791   2.2434  2.5760 0.0106  1.3609 10.1973
age                    0.0414   0.1875  0.2211 0.8252 -0.3278  0.4107
Medu                   0.2372   0.2456  0.9657 0.3351 -0.2465  0.7208
Fedu                   0.3416   0.2215  1.5420 0.1243 -0.0947  0.7778
traveltime            -0.0312   0.2934 -0.1065 0.9153 -0.6091  0.5466
studytime              0.4362   0.2434  1.7922 0.0743 -0.0431  0.9156
failures              -0.9865   0.2859 -3.4508 0.0007 -1.5495 -0.4235
famrel                 0.0930   0.2123  0.4380 0.6618 -0.3251  0.5110
freetime               0.3212   0.1964  1.6356 0.1032 -0.0655  0.7079
goout                 -0.7147   0.2013 -3.5503 0.0005 -1.1112 -0.3183
Dalc                  -0.1118   0.2801 -0.3991 0.6901 -0.6634  0.4398
Walc                   0.1374   0.2098  0.6546 0.5133 -0.2759  0.5506
health                -0.2397   0.1423 -1.6840 0.0934 -0.5200  0.0406
absences              -0.0103   0.0245 -0.4202 0.6747 -0.0584  0.0379
is_Male                0.5097   0.4338  1.1750 0.2411 -0.3447  1.3641
is_school_MS          -0.4051   0.6657 -0.6085 0.5434 -1.7163  0.9060
is_address_R          -0.1615   0.4881 -0.3309 0.7410 -1.1228  0.7997
is_famsize_LE3         0.4223   0.4358  0.9690 0.3335 -0.4360  1.2805
is_Pstatus_A           0.0600   0.6488  0.0925 0.9264 -1.2178  1.3378
is_schoolsup           2.0342   1.0979  1.8528 0.0651 -0.1280  4.1964
not_famsup             0.8491   0.4158  2.0420 0.0422  0.0302  1.6681
is_paid               -0.2096   0.4007 -0.5232 0.6013 -0.9987  0.5795
not_activities         0.4959   0.3710  1.3366 0.1826 -0.2348  1.2265
not_nursery            0.0221   0.4608  0.0479 0.9618 -0.8854  0.9296
not_higher            -1.1088   0.8891 -1.2472 0.2135 -2.8598  0.6421
not_internet           3.7449   1.2133  3.0865 0.0023  1.3554  6.1344
is_romantic            0.0002   0.4008  0.0005 0.9996 -0.7892  0.7896
not_Mjob               0.2646   0.5740  0.4609 0.6453 -0.8659  1.3950
not_Fjob               0.9844   0.9733  1.0114 0.3128 -0.9325  2.9013
is_reason_reputation   0.0328   0.7148  0.0459 0.9635 -1.3749  1.4404
is_reason_course      -0.6429   0.6742 -0.9535 0.3412 -1.9707  0.6849
is_reason_home        -0.1942   0.7120 -0.2728 0.7853 -1.5965  1.2081
is_guardian_notparent  0.1790   0.7835  0.2285 0.8194 -1.3639  1.7220
---------------------------------------------------------------------
Omnibus:            5.982     Durbin-Watson:        2.032            
Prob(Omnibus):      0.050     Jarque-Bera (JB):     4.009            
Skew:               0.130     Prob(JB):             0.135            
Kurtosis:           2.479     Condition No.:        25897555596586012
=====================================================================
* The condition number is large (3e+16). This might indicate
strong multicollinearity or other numerical problems.
"""

In [136]:
#Modelo con todos los regresores
modelo_saturadog2 = smf.ols('G2 ~ '+v_independientes, data=df)
modelo_saturadog2 = modelo_saturadog2.fit()
modelo_saturadog2.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Results: Ordinary least squares
=====================================================================
Model:               OLS               Adj. R-squared:      0.173    
Dependent Variable:  G2                AIC:                 1538.8329
Date:                2022-11-23 05:33  BIC:                 1655.6000
No. Observations:    284               Log-Likelihood:      -737.42  
Df Model:            31                F-statistic:         2.906    
Df Residuals:        252               Prob (F-statistic):  2.18e-06 
R-squared:           0.263             Scale:               11.879   
---------------------------------------------------------------------
                       Coef.  Std.Err.    t    P>|t|   [0.025  0.975]
---------------------------------------------------------------------
Intercept              9.7054   2.6037  3.7276 0.0002  4.5777 14.8332
age                   -0.1378   0.2176 -0.6335 0.5270 -0.5664  0.2907
Medu                   0.2911   0.2850  1.0215 0.3080 -0.2702  0.8524
Fedu                   0.0423   0.2571  0.1645 0.8695 -0.4640  0.5486
traveltime            -0.2601   0.3405 -0.7637 0.4458 -0.9307  0.4106
studytime              0.4011   0.2825  1.4199 0.1569 -0.1552  0.9575
failures              -0.9978   0.3318 -3.0074 0.0029 -1.6512 -0.3444
famrel                 0.0203   0.2463  0.0824 0.9344 -0.4649  0.5055
freetime               0.2582   0.2279  1.1330 0.2583 -0.1906  0.7071
goout                 -0.8982   0.2336 -3.8443 0.0002 -1.3584 -0.4381
Dalc                  -0.1956   0.3251 -0.6016 0.5480 -0.8358  0.4446
Walc                   0.3369   0.2435  1.3834 0.1678 -0.1427  0.8165
health                -0.3597   0.1652 -2.1779 0.0303 -0.6850 -0.0344
absences              -0.0082   0.0284 -0.2892 0.7727 -0.0641  0.0477
is_Male                0.6926   0.5035  1.3755 0.1702 -0.2990  1.6842
is_school_MS           0.3426   0.7727  0.4434 0.6578 -1.1791  1.8643
is_address_R          -0.9885   0.5665 -1.7449 0.0822 -2.1041  0.1272
is_famsize_LE3         0.5060   0.5058  1.0004 0.3181 -0.4901  1.5021
is_Pstatus_A           0.3474   0.7530  0.4613 0.6450 -1.1357  1.8304
is_schoolsup           4.3494   1.2742  3.4135 0.0007  1.8400  6.8588
not_famsup             0.8654   0.4826  1.7930 0.0742 -0.0851  1.8158
is_paid                0.2622   0.4650  0.5638 0.5734 -0.6537  1.1780
not_activities         0.1709   0.4306  0.3970 0.6917 -0.6771  1.0189
not_nursery           -0.1272   0.5348 -0.2378 0.8122 -1.1804  0.9261
not_higher            -1.0229   1.0318 -0.9914 0.3224 -3.0551  1.0092
not_internet           5.3560   1.4081  3.8036 0.0002  2.5828  8.1292
is_romantic           -0.5937   0.4652 -1.2762 0.2031 -1.5099  0.3225
not_Mjob              -0.2051   0.6662 -0.3079 0.7584 -1.5171  1.1069
not_Fjob               0.1143   1.1296  0.1012 0.9195 -2.1104  2.3390
is_reason_reputation  -0.4465   0.8295 -0.5382 0.5909 -2.0802  1.1872
is_reason_course      -1.2881   0.7825 -1.6462 0.1010 -2.8291  0.2529
is_reason_home        -0.7287   0.8264 -0.8818 0.3787 -2.3562  0.8988
is_guardian_notparent  0.0871   0.9093  0.0958 0.9238 -1.7036  1.8778
---------------------------------------------------------------------
Omnibus:           11.384     Durbin-Watson:        1.829            
Prob(Omnibus):     0.003      Jarque-Bera (JB):     12.145           
Skew:              -0.417     Prob(JB):             0.002            
Kurtosis:          3.576      Condition No.:        25897555596586012
=====================================================================
* The condition number is large (3e+16). This might indicate
strong multicollinearity or other numerical problems.
"""

In [51]:
#R^2 en ambos casos es similar al G3, el poder explicativo para cada variables es muy bajo con el modelo satutado (todas las variables)
# Las variables regresoras se obtiene que failures, goout, not_internet son nuevamente significativas
# para G1 y G2, mientras que  y is_schoolsup es solo para G2 y G3, en cuaanto a Walc, solo es significativa para G3
# La variable not_famsup es regresora para G1 y health para G2

In [137]:
#Para G3, se seleccionan las variables failures, goout,Walc, is_schoolsup y not_internet

In [138]:
var_regresorasg3 = ['failures', 'goout', 'Walc', 'is_schoolsup' , 'not_internet']
var_regresorasg3

['failures', 'goout', 'Walc', 'is_schoolsup', 'not_internet']

In [139]:
variables_independientesg3 = "+".join(var_regresorasg3)
variables_independientesg3

'failures+goout+Walc+is_schoolsup+not_internet'

In [140]:
modelo_depuradog3 = smf.ols('G3 ~ '+variables_independientesg3, data=df)
modelo_depuradog3 = modelo_depuradog3.fit()
modelo_depuradog3.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.155    
Dependent Variable: G3               AIC:                1625.7479
Date:               2022-11-23 05:33 BIC:                1643.9928
No. Observations:   284              Log-Likelihood:     -807.87  
Df Model:           4                F-statistic:        13.95    
Df Residuals:       279              Prob (F-statistic): 2.18e-10 
R-squared:          0.167            Scale:              17.622   
-------------------------------------------------------------------
                Coef.   Std.Err.     t     P>|t|    [0.025   0.975]
-------------------------------------------------------------------
Intercept       8.4302    0.5335  15.8030  0.0000   7.3801   9.4803
failures       -2.0335    0.3477  -5.8492  0.0000  -2.7179  -1.3491
goout          -0.9455    0.2616  -3.6149  0.0004  -1.4605  -0.4306
Walc            0.3967    0.2188   1.8129  0.0709  -0.0341   0.8275
is_schoolsup    3.5906    0.5109   7.0277  0.0000   2.5848   4.5963
not_internet    4.8396    0.3877  12.4833  0.0000   4.0765   5.6028
------------------------------------------------------------------
Omnibus:          21.095    Durbin-Watson:       1.854            
Prob(Omnibus):    0.000     Jarque-Bera (JB):    23.804           
Skew:             -0.646    Prob(JB):            0.000            
Kurtosis:         3.584     Condition No.:       67617693823674888
==================================================================
* The condition number is large (7e+16). This might indicate
strong multicollinearity or other numerical problems.
"""

In [141]:
#Nuevamente el R^2 es bajo (16%). La variable Walc al parecer no es regresora adecuada en el modelo, se probará nuevamente
#sacando  esta variable

In [142]:
var_regresoras3 = ['failures', 'goout', 'is_schoolsup' , 'not_internet']
var_regresoras3

['failures', 'goout', 'is_schoolsup', 'not_internet']

In [143]:
variables_independientes3 = "+".join(var_regresoras3)
variables_independientes3

'failures+goout+is_schoolsup+not_internet'

In [144]:
modelo_depuradog3_2 = smf.ols('G3 ~ '+variables_independientes3, data=df)
modelo_depuradog3_2 = modelo_depuradog3_2.fit()
modelo_depuradog3_2.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.148    
Dependent Variable: G3               AIC:                1627.0738
Date:               2022-11-23 05:34 BIC:                1641.6696
No. Observations:   284              Log-Likelihood:     -809.54  
Df Model:           3                F-statistic:        17.36    
Df Residuals:       280              Prob (F-statistic): 2.28e-10 
R-squared:          0.157            Scale:              17.766   
-------------------------------------------------------------------
                Coef.   Std.Err.     t     P>|t|    [0.025   0.975]
-------------------------------------------------------------------
Intercept       8.5683    0.5301  16.1622  0.0000   7.5247   9.6118
failures       -1.9757    0.3476  -5.6838  0.0000  -2.6600  -1.2915
goout          -0.7387    0.2363  -3.1256  0.0020  -1.2039  -0.2735
is_schoolsup    3.6269    0.5126   7.0755  0.0000   2.6179   4.6360
not_internet    4.9413    0.3852  12.8291  0.0000   4.1832   5.6995
------------------------------------------------------------------
Omnibus:          28.463    Durbin-Watson:       1.849            
Prob(Omnibus):    0.000     Jarque-Bera (JB):    34.594           
Skew:             -0.763    Prob(JB):            0.000            
Kurtosis:         3.770     Condition No.:       36100168918681256
==================================================================
* The condition number is large (4e+16). This might indicate
strong multicollinearity or other numerical problems.
"""

In [145]:
#El modelo sin Walc bajó levemente su R^2, puede que esta variable aporte realmente a al modelo predictor. 

#Se estimará el modelo depurado para G1 y G2 con sus regresores 

In [146]:
var_regresorasg1 = ['failures', 'goout', 'not_famsup', 'not_internet']
var_regresorasg1

['failures', 'goout', 'not_famsup', 'not_internet']

In [147]:
variables_independientesg1 = "+".join(var_regresorasg1)
variables_independientesg1

'failures+goout+not_famsup+not_internet'

In [92]:
modelo_depuradog1 = smf.ols('G1 ~ '+variables_independientesg1, data=df)
modelo_depuradog1 = modelo_depuradog1.fit()
modelo_depuradog1.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.180    
Dependent Variable: G1               AIC:                1430.7433
Date:               2022-11-23 05:31 BIC:                1448.9882
No. Observations:   284              Log-Likelihood:     -710.37  
Df Model:           4                F-statistic:        16.56    
Df Residuals:       279              Prob (F-statistic): 3.45e-12 
R-squared:          0.192            Scale:              8.8685   
-------------------------------------------------------------------
                Coef.   Std.Err.     t     P>|t|    [0.025   0.975]
-------------------------------------------------------------------
Intercept      11.2439    0.6888  16.3229  0.0000   9.8879  12.5999
failures       -1.3469    0.2457  -5.4815  0.0000  -1.8306  -0.8632
goout          -0.5755    0.1670  -3.4466  0.0007  -0.9042  -0.2468
not_famsup      0.5871    0.3667   1.6011  0.1105  -0.1347   1.3090
not_internet    1.9481    0.5251   3.7097  0.0003   0.9144   2.9819
------------------------------------------------------------------
Omnibus:               6.832        Durbin-Watson:           1.976
Prob(Omnibus):         0.033        Jarque-Bera (JB):        5.013
Skew:                  0.202        Prob(JB):                0.082
Kurtosis:              2.490        Condition No.:           16   
==================================================================

"""

In [93]:
# Se puede descartar a not_famsup como regresor de G1, R^ 19%

In [148]:
var_regresorasg2 = ['failures', 'goout', 'health', 'is_schoolsup' , 'not_internet']
var_regresorasg2

['failures', 'goout', 'health', 'is_schoolsup', 'not_internet']

In [149]:
variables_independientesg2 = "+".join(var_regresorasg2)
variables_independientesg2

'failures+goout+health+is_schoolsup+not_internet'

In [150]:
modelo_depuradog2 = smf.ols('G2 ~ '+variables_independientesg2, data=df)
modelo_depuradog2 = modelo_depuradog2.fit()
modelo_depuradog2.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.154    
Dependent Variable: G2               AIC:                1520.0935
Date:               2022-11-23 05:34 BIC:                1538.3384
No. Observations:   284              Log-Likelihood:     -755.05  
Df Model:           4                F-statistic:        13.88    
Df Residuals:       279              Prob (F-statistic): 2.46e-10 
R-squared:          0.166            Scale:              12.147   
-------------------------------------------------------------------
                Coef.   Std.Err.     t     P>|t|    [0.025   0.975]
-------------------------------------------------------------------
Intercept       9.3804    0.5793  16.1935  0.0000   8.2401  10.5207
failures       -1.4457    0.2886  -5.0090  0.0000  -2.0139  -0.8776
goout          -0.6715    0.1954  -3.4360  0.0007  -1.0562  -0.2868
health         -0.3704    0.1577  -2.3483  0.0196  -0.6809  -0.0599
is_schoolsup    4.0506    0.4623   8.7609  0.0000   3.1405   4.9607
not_internet    5.3298    0.3729  14.2917  0.0000   4.5957   6.0639
------------------------------------------------------------------
Omnibus:           18.409   Durbin-Watson:      1.813             
Prob(Omnibus):     0.000    Jarque-Bera (JB):   21.892            
Skew:              -0.538   Prob(JB):           0.000             
Kurtosis:          3.833    Condition No.:      421045951217343552
==================================================================
* The condition number is large (4e+17). This might indicate
strong multicollinearity or other numerical problems.
"""

In [149]:
# Todas las variables regresoras son significativas para G2, R^2= 16%

In [151]:
# Las variables regresoras en común para las variables dependientes son: failures, goout y not_internet
# Los modelo de regresión obtenidos para los tres casos explica muy poco de las variables G1, G2 y G3 según su R cuadrado,
# repectivamente. Puede ser que no se encuentre información suficiente para determinar el comportamiento de la variable

#### Modelación predictiva.

In [152]:
# Como el objetivo es determinar un modelo predictivo para el desempeño académico (G3) mediante el entrenamiento de un modelo
# de regresión lineal, se toma como variable y a G3

In [153]:
y= df['G3']
X = df[v_no_categoricas]


In [154]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1121)

In [155]:
X_train.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,not_nursery,not_higher,not_internet,is_romantic,not_Mjob,not_Fjob,is_reason_reputation,is_reason_course,is_reason_home,is_guardian_notparent
161,15,3,2,2,2,2,4,4,4,1,...,0,0,0,1,0,0,0,1,0,0
63,16,4,3,1,3,0,3,4,4,2,...,0,0,0,0,0,0,0,0,1,0
264,18,2,2,1,3,0,4,3,3,1,...,0,0,1,1,1,0,0,0,1,0
197,16,3,3,3,1,0,3,3,4,3,...,0,0,1,0,0,0,0,0,1,0
86,16,2,2,1,2,0,4,3,4,1,...,0,0,1,0,1,0,0,1,0,0


In [156]:
y_train.head()

161     7
63      9
264     0
197    10
86      6
Name: G3, dtype: int64

In [157]:
#Ahora generamos el modelo con intercepto

In [158]:
#Con intercepto
#Instanciamos el modelo
lm_1 = linear_model.LinearRegression(fit_intercept=True)
#Entrenamos
lm_1.fit(X_train, y_train)
#Predecimos en  test
#Prediccion de la matriz de prueba
predictions_test_1 = lm_1.predict(X_test)
#predictions_test_1

In [159]:
def report_scores(prediction_test, vector):
    """Cálculo del error cuadratico medio y R cuadrado para un modelo de regresión lineal """
    #Realizamos métricas 
    mse=mean_squared_error(vector, prediction_test).round(2)
    r2=r2_score(vector, prediction_test).round(2)
    print(f'Error cuadrático medio: ', mse, 'R2: ', r2)

In [160]:
#Modelo con intercepto
report_scores(predictions_test_1, y_test)

Error cuadrático medio:  20.46 R2:  0.03


In [161]:
# Ahora observaremos las correlaciones de las variables independientes con G3

In [162]:
v_cat = []
for n, i in enumerate(df):
    if df[i].dtypes =="object":
        #print(df[i].dtypes)
        #print(df[i])
        v_cat.append(i)

In [163]:
v_cat

['school',
 'sex',
 'address',
 'famsize',
 'Pstatus',
 'Mjob',
 'Fjob',
 'reason',
 'guardian',
 'schoolsup',
 'famsup',
 'paid',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic']

In [164]:
df=df[df.columns[~df.columns.isin(['school','sex','address','famsize','Pstatus','Mjob','Fjob','reason','guardian','schoolsup','famsup','paid','activities','nursery','higher','internet','romantic'])]]

In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 1 to 394
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   age                    284 non-null    int64
 1   Medu                   284 non-null    int64
 2   Fedu                   284 non-null    int64
 3   traveltime             284 non-null    int64
 4   studytime              284 non-null    int64
 5   failures               284 non-null    int64
 6   famrel                 284 non-null    int64
 7   freetime               284 non-null    int64
 8   goout                  284 non-null    int64
 9   Dalc                   284 non-null    int64
 10  Walc                   284 non-null    int64
 11  health                 284 non-null    int64
 12  absences               284 non-null    int64
 13  G1                     284 non-null    int64
 14  G2                     284 non-null    int64
 15  G3                     284 non-null    i

In [166]:
def fetch_features(df, vector):
        """Correlaciones de variables del dataframe con variable objetivo """
        # nombres de las columnas 
        columnas = df.columns
        # array para nombre de la variable
        attr_name = []
        # array para correlación de pearson
        pearson_r = []
        # array para valor absoluto de la correlación
        abs_pearson_r = []
    
        for c in columnas:
        # si la columna no es la dependiente
            if c != vector:
                # adjuntar el nombre de la variable en attr_name
                attr_name.append(c)
                # adjuntar la correlación de pearson
                pearson_r.append(df[c].corr(df[vector]))
                # adjuntar el absoluto de la correlación de pearson
                abs_pearson_r.append(abs(df[c].corr(df[vector])))
            
        # El arrays se transforma en DataFrame
        features = pd.DataFrame({
        'attribute': attr_name,
        'corr':pearson_r,
        'abs_corr':abs_pearson_r
        })
        # Se crea el index con los nombres de las variables
        features = features.set_index('attribute')
        # ordenamos los valores de forma descendiente
        features=features.sort_values(by=['abs_corr'], ascending=False)
        return features

In [167]:
fetch_features(df,'G3')

,corr,abs_corr
attribute,,
G2,0.902984,0.902984
G1,0.803384,0.803384
failures,-0.345628,0.345628
goout,-0.221752,0.221752
Medu,0.180326,0.180326
not_higher,-0.162044,0.162044
is_reason_course,-0.140458,0.140458
age,-0.132134,0.132134
is_address_R,-0.119397,0.119397


In [168]:
#Las 6 mayores correlaciones son:
correc=fetch_features(df, 'G3')
correc.head(6) #se toman las primeras 6 porque la función está ordenada de forma desendente por asb_corr

,corr,abs_corr
attribute,,
G2,0.902984,0.902984
G1,0.803384,0.803384
failures,-0.345628,0.345628
goout,-0.221752,0.221752
Medu,0.180326,0.180326
not_higher,-0.162044,0.162044


In [169]:
# G1 Y G2 evidentmente estarán correlacionadas ya que con ambas se saca el promedio correspondiente a G3, no se consideran
# Las variables más correlacionadas son failures, goout y Medu (se corta en 18% debido a las bajas correlaciones), aunque 
#su correlacion es muy baja (menor a 50%)

# Estas correlaciones bajas coincide con los modelos de bajos poder explicativo obtenidos con el punto anterior
#(Modelo sdescriptivo). 

In [170]:
#Se generará otro conjunto de entrenamiento con estas variables obtenidas por correlación y se compararan con 
#intercepto y sin intercepto

In [171]:
var=['failures', 'goout', 'Medu']
var



['failures', 'goout', 'Medu']

In [172]:
var_3=df.loc[:,var]
var_3

,failures,goout,Medu
1,0,3,1
3,0,2,4
5,0,2,4
6,0,4,2
7,0,4,4
...,...,...,...
390,2,4,2
391,0,5,3
392,3,3,1
393,0,1,3


In [173]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(var_3, y, test_size=0.33, random_state=1121)

In [174]:
#Con intercepto
#Instanciamos el modelo
lm_3 = linear_model.LinearRegression(fit_intercept=True)
#Entrenamos
lm_3.fit(X_train2, y_train2)
#Predecimos en  test
#Prediccion de la matriz de prueba
predictions_test_3 = lm_3.predict(X_test2)

In [175]:
report_scores(predictions_test_3, y_test2)

Error cuadrático medio:  17.84 R2:  0.15


In [176]:
#Son intercepto
#Instanciamos el modelo
lm_4 = linear_model.LinearRegression(fit_intercept=False)
#Entrenamos
lm_4.fit(X_train2, y_train2)
#Predecimos en  test
#Prediccion de la matriz de prueba
predictions_test_4 = lm_4.predict(X_test2)
#predictions_test_4

In [177]:
report_scores(predictions_test_4, y_test2)

Error cuadrático medio:  26.38 R2:  -0.25


In [178]:
# El modelo predictivo para G3 con todas las variables numéricas obtuvos los siguientes resultados:
##Error cuadrático medio:  20.46 R2:  0.03

# El modelo predictivo para G3 con solo las variables 'failures', 'goout', 'Medu', con intercepto obtuvo lo siguiente:
## Error cuadrático medio:  17.84 R2:  0.15

## El modelo predictivo para G3 con solo las variables 'failures', 'goout', 'Medu', sin intercepto obtuvo lo siguiente:
## Error cuadrático medio:  26.38 R2:  -0.25


# Los modelos con las variables más correlacionadas obtuvieron mejores R^2, que indican que tienen un mayor poder
# explicativo de la variable G3. El error cuadrático medio se dispara con el modelo sin intercepto. Se considera que 
# El modelo predictivo con intercepto es mejor ya que aumenta el R^2 y disminuye el error cuadratico medio.

# Finalmente como comentario, la base de información es muy importante para obtener resultados optimos. Si no se 
# recolecta la información de forma ordenada y prolija esto puede generar gran perdida de información e impedir
# sacar información correcta de los datos. Puede ser que la estimación de los modelos predictivos y descriptivos se vieran
# afectados por la baja cantidad de datos que finalmente quedaron al tener que limpiar la base de datos. 